In [1]:
import os
import glob
import napari
import pandas as pd
import numpy as np
#import pyclesperanto_prototype as cle

from pathlib import Path
from aicsimageio import AICSImage, readers
from blimp.processing.segment_and_quantify import segment_nuclei_cellpose, quantify
from blimp.preprocessing.illumination_correction import IlluminationCorrection

data_dir = "/srv/scratch/berrylab/z3532965/systems_Ti2/202402_BleachChase_mCherry-POLR2A/"
image_dir = os.path.join(data_dir,"20240208_085709_941/OME-TIFF-MIP")
image_dir_3D = os.path.join(data_dir,"20240208_085709_941/OME-TIFF")

ana_dir = "/srv/scratch/z3532965/src/blana/Scott/202402_BleachChase_mCherry-POLR2A"
illum_corr_file = os.path.join(data_dir,"ILLUMCORR/illumination_correction.pkl")

03-Mar-24 15:27:31 - bfio.backends - WARNING  - Java backend is not available. This could be due to a missing dependency (jpype).


# Load metadata
We have a csv file of the plate layout that includes information on antibody staining

In [2]:
plate_layout = pd.read_csv(os.path.join(ana_dir,"METADATA","plate_layout.csv"))
plate_layout.columns

Index(['well_name', 'siRNA', 'replicate'], dtype='object')

# Apply illumination correction

Load illumination correction object

In [4]:
illumination_correction = IlluminationCorrection(from_file=illum_corr_file)

In [9]:
image_metadata_files = glob.glob(os.path.join(image_dir,"*.csv"))
image_metadata = pd.concat((pd.read_csv(f) for f in image_metadata_files), ignore_index=True)
image_metadata['well_name'] = image_metadata['filename_ome_tiff'].str.extract(r'Well([A-Z]\d{2})_Channel')
image_metadata['channel'] = image_metadata['filename_ome_tiff'].str.extract(r'Channel(TRITC|Red)')
image_metadata.columns

Index(['n_pixels_y', 'n_pixels_x', 'objective_name', 'pixel_size_microns',
       'field_id', 'timepoint_id', 'filename_ome_tiff', 'acquisition_time_rel',
       'stage_y_abs', 'stage_x_abs', 'stage_z_n', 'acquisition_time_abs',
       'standard_field_id', 'metadata_string_acquisition_0', 'well_name',
       'channel'],
      dtype='object')

In [10]:
image_metadata_annotated = image_metadata.merge(plate_layout,left_on="well_name", right_on="well_name")

In [11]:
image_metadata_annotated.query("siRNA=='ARMC5' & replicate == 1 & field_id == 1")

,n_pixels_y,n_pixels_x,objective_name,pixel_size_microns,field_id,timepoint_id,filename_ome_tiff,acquisition_time_rel,stage_y_abs,stage_x_abs,stage_z_n,acquisition_time_abs,standard_field_id,metadata_string_acquisition_0,well_name,channel,siRNA,replicate
0,2304,2304,Plan Apo VC 20x DIC N2,0.325,1,0,WellD06_ChannelRed_Scott_Seq0005_0001.ome.tiff,4912.295424,-5616.985714,3204.700000,7,2024-08-02 12:18:59.295424,3,Metadata:\nDimensions: T(4) x XY(9) x Z(7) x λ...,D06,Red,ARMC5,1
9,2304,2304,Plan Apo VC 20x DIC N2,0.325,1,1,WellD06_ChannelRed_Scott_Seq0005_0001.ome.tiff,5571.710209,-5617.171429,3204.600000,7,2024-08-02 12:29:58.710209,3,Metadata:\nDimensions: T(4) x XY(9) x Z(7) x λ...,D06,Red,ARMC5,1
18,2304,2304,Plan Apo VC 20x DIC N2,0.325,1,2,WellD06_ChannelRed_Scott_Seq0005_0001.ome.tiff,6231.221320,-5617.285714,3204.614286,7,2024-08-02 12:40:58.221320,3,Metadata:\nDimensions: T(4) x XY(9) x Z(7) x λ...,D06,Red,ARMC5,1
27,2304,2304,Plan Apo VC 20x DIC N2,0.325,1,3,WellD06_ChannelRed_Scott_Seq0005_0001.ome.tiff,6890.685189,-5617.100000,3204.600000,7,2024-08-02 12:51:57.685189,3,Metadata:\nDimensions: T(4) x XY(9) x Z(7) x λ...,D06,Red,ARMC5,1
36,2304,2304,Plan Apo VC 20x DIC N2,0.325,1,0,WellD06_ChannelTRITC_Seq0003_0001.ome.tiff,200.687898,-5617.142857,6004.500000,7,2024-08-02 10:16:31.687898,3,Metadata:\nDimensions: T(6) x XY(9) x Z(7) x λ...,D06,TRITC,ARMC5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
783,2304,2304,Plan Apo VC 20x DIC N2,0.325,1,1,WellD06_ChannelTRITC_Seq0001_0001.ome.tiff,930.540839,-5616.985714,3204.614286,7,2024-08-02 10:26:30.540839,3,Metadata:\nDimensions: T(6) x XY(9) x Z(7) x λ...,D06,TRITC,ARMC5,1
792,2304,2304,Plan Apo VC 20x DIC N2,0.325,1,2,WellD06_ChannelTRITC_Seq0001_0001.ome.tiff,1791.316360,-5617.128571,3204.800000,7,2024-08-02 10:40:51.316360,3,Metadata:\nDimensions: T(6) x XY(9) x Z(7) x λ...,D06,TRITC,ARMC5,1
801,2304,2304,Plan Apo VC 20x DIC N2,0.325,1,3,WellD06_ChannelTRITC_Seq0001_0001.ome.tiff,2651.317142,-5617.057143,3204.514286,7,2024-08-02 10:55:11.317142,3,Metadata:\nDimensions: T(6) x XY(9) x Z(7) x λ...,D06,TRITC,ARMC5,1
810,2304,2304,Plan Apo VC 20x DIC N2,0.325,1,4,WellD06_ChannelTRITC_Seq0001_0001.ome.tiff,3511.711520,-5617.185714,3204.800000,7,2024-08-02 11:09:31.711520,3,Metadata:\nDimensions: T(6) x XY(9) x Z(7) x λ...,D06,TRITC,ARMC5,1


In [12]:
image_metadata_annotated.query("siRNA=='ARMC5' & replicate == 1 & timepoint_id == 1")

,n_pixels_y,n_pixels_x,objective_name,pixel_size_microns,field_id,timepoint_id,filename_ome_tiff,acquisition_time_rel,stage_y_abs,stage_x_abs,stage_z_n,acquisition_time_abs,standard_field_id,metadata_string_acquisition_0,well_name,channel,siRNA,replicate
9,2304,2304,Plan Apo VC 20x DIC N2,0.325,1,1,WellD06_ChannelRed_Scott_Seq0005_0001.ome.tiff,5571.710209,-5617.171429,3204.600000,7,2024-08-02 12:29:58.710209,3,Metadata:\nDimensions: T(4) x XY(9) x Z(7) x λ...,D06,Red,ARMC5,1
10,2304,2304,Plan Apo VC 20x DIC N2,0.325,2,1,WellD06_ChannelRed_Scott_Seq0005_0002.ome.tiff,5608.132617,-5622.700000,2455.600000,7,2024-08-02 12:30:35.132617,2,Metadata:\nDimensions: T(4) x XY(9) x Z(7) x λ...,D06,Red,ARMC5,1
11,2304,2304,Plan Apo VC 20x DIC N2,0.325,3,1,WellD06_ChannelRed_Scott_Seq0005_0003.ome.tiff,5644.499185,-5628.300000,1707.000000,7,2024-08-02 12:31:11.499185,1,Metadata:\nDimensions: T(4) x XY(9) x Z(7) x λ...,D06,Red,ARMC5,1
12,2304,2304,Plan Apo VC 20x DIC N2,0.325,4,1,WellD06_ChannelRed_Scott_Seq0005_0004.ome.tiff,5680.757341,-4879.671429,1700.771429,7,2024-08-02 12:31:47.757341,4,Metadata:\nDimensions: T(4) x XY(9) x Z(7) x λ...,D06,Red,ARMC5,1
13,2304,2304,Plan Apo VC 20x DIC N2,0.325,5,1,WellD06_ChannelRed_Scott_Seq0005_0005.ome.tiff,5717.200793,-4873.700000,2449.785714,7,2024-08-02 12:32:24.200793,5,Metadata:\nDimensions: T(4) x XY(9) x Z(7) x λ...,D06,Red,ARMC5,1
14,2304,2304,Plan Apo VC 20x DIC N2,0.325,6,1,WellD06_ChannelRed_Scott_Seq0005_0006.ome.tiff,5753.658590,-4868.100000,3198.871429,7,2024-08-02 12:33:00.658590,6,Metadata:\nDimensions: T(4) x XY(9) x Z(7) x λ...,D06,Red,ARMC5,1
15,2304,2304,Plan Apo VC 20x DIC N2,0.325,7,1,WellD06_ChannelRed_Scott_Seq0005_0007.ome.tiff,5789.864607,-4119.314286,3193.185714,7,2024-08-02 12:33:36.864607,9,Metadata:\nDimensions: T(4) x XY(9) x Z(7) x λ...,D06,Red,ARMC5,1
16,2304,2304,Plan Apo VC 20x DIC N2,0.325,8,1,WellD06_ChannelRed_Scott_Seq0005_0008.ome.tiff,5826.268271,-4125.000000,2444.300000,7,2024-08-02 12:34:13.268271,8,Metadata:\nDimensions: T(4) x XY(9) x Z(7) x λ...,D06,Red,ARMC5,1
17,2304,2304,Plan Apo VC 20x DIC N2,0.325,9,1,WellD06_ChannelRed_Scott_Seq0005_0009.ome.tiff,5862.741090,-4130.700000,1695.614286,7,2024-08-02 12:34:49.741090,7,Metadata:\nDimensions: T(4) x XY(9) x Z(7) x λ...,D06,Red,ARMC5,1
45,2304,2304,Plan Apo VC 20x DIC N2,0.325,1,1,WellD06_ChannelTRITC_Seq0003_0001.ome.tiff,1061.161528,-5617.157143,6004.400000,7,2024-08-02 10:30:52.161528,3,Metadata:\nDimensions: T(6) x XY(9) x Z(7) x λ...,D06,TRITC,ARMC5,1


## Save Examples

In [20]:
import random
random.seed(10)
example_images = image_metadata_annotated.query("channel=='TRITC'").groupby(['siRNA']).apply(pd.DataFrame.sample,n=2)

In [21]:
example_images

n_pixels_y  n_pixels_x          objective_name  \
siRNA                                                            
ARMC5     1064        2304        2304  Plan Apo VC 20x DIC N2   
          981         2304        2304  Plan Apo VC 20x DIC N2   
Scrambled 2552        2304        2304  Plan Apo VC 20x DIC N2   
          2895        2304        2304  Plan Apo VC 20x DIC N2   

                pixel_size_microns  field_id  timepoint_id  \
siRNA                                                        
ARMC5     1064               0.325         3            16   
          981                0.325         1             7   
Scrambled 2552               0.325         6             3   
          2895               0.325         7            35   

                                         filename_ome_tiff  \
siRNA                                                        
ARMC5     1064  WellE05_ChannelTRITC_Seq0016_0003.ome.tiff   
          981   WellE05_ChannelTRITC_Seq0016_0001.ome.tiff   
Scrambled 2552  WellE06_ChannelTRITC_Seq0011_0006.ome.tiff   
          2895  WellE06_ChannelTRITC_Seq0019_0007.ome.tiff   

                acquisition_time_rel  stage_y_abs   stage_x_abs  stage_z_n  \
siRNA                                                                        
ARMC5     1064          60937.568508  3371.800000  10614.600000          7   
          981           53161.183196  3382.742857  12112.400000          7   
Scrambled 2552          42592.469372  4126.085714   3163.914286          7   
          2895          77546.771691  4874.528571   5958.514286          7   

                      acquisition_time_abs  standard_field_id  \
siRNA                                                           
ARMC5     1064  2024-09-02 23:50:59.568508                  1   
          981   2024-09-02 21:41:23.183196                  3   
Scrambled 2552  2024-08-03 09:06:02.469372                  6   
          2895  2024-09-03 04:31:04.771691                  9   

                                    metadata_string_acquisition_0 well_name  \
siRNA                                                                         
ARMC5     1064  Metadata:\nDimensions: T(38) x XY(9) x Z(7) x ...       E05   
          981   Metadata:\nDimensions: T(38) x XY(9) x Z(7) x ...       E05   
Scrambled 2552  Metadata:\nDimensions: T(6) x XY(9) x Z(7) x λ...       E06   
          2895  Metadata:\nDimensions: T(38) x XY(9) x Z(7) x ...       E06   

               channel      siRNA  replicate  
siRNA                                         
ARMC5     1064   TRITC      ARMC5          2  
          981    TRITC      ARMC5          2  
Scrambled 2552   TRITC  Scrambled          2  
          2895   TRITC  Scrambled          2

## Save examples for training nuclear segmentation model

Here we crop images before saving to remove vignetting artefact and ensure training data is appropriate

In [22]:
training_dir = Path(image_dir).parent / 'TRAINING'
if not training_dir.exists(): 
    training_dir.mkdir()

In [25]:
for index, row in example_images.iterrows():
    aics_image = AICSImage(Path(image_dir) / row["filename_ome_tiff"], reader=readers.ome_tiff_reader.OmeTiffReader)
    mCherry_array = aics_image.get_image_data('YX',Z=0,C=0,T=row["timepoint_id"])
    mCherry_image = illumination_correction.correct(AICSImage(mCherry_array[np.newaxis,np.newaxis,np.newaxis,:,:]))
    # crop 500 pixels from left edge
    mCherry_image = AICSImage(mCherry_image.get_image_data('YX',Z=0,C=0,T=0)[np.newaxis,np.newaxis,np.newaxis,:,500:])
    mCherry_image.save(str(training_dir / row["filename_ome_tiff"]))

Get also the lowest intensity images (immediately after bleach to ensure these are properly segmented

In [26]:
image_metadata_annotated.query("channel=='TRITC' & replicate == 1 & timepoint_id == 1 & field_id == 1")

,n_pixels_y,n_pixels_x,objective_name,pixel_size_microns,field_id,timepoint_id,filename_ome_tiff,acquisition_time_rel,stage_y_abs,stage_x_abs,stage_z_n,acquisition_time_abs,standard_field_id,metadata_string_acquisition_0,well_name,channel,siRNA,replicate
45,2304,2304,Plan Apo VC 20x DIC N2,0.325,1,1,WellD06_ChannelTRITC_Seq0003_0001.ome.tiff,1061.161528,-5617.157143,6004.400000,7,2024-08-02 10:30:52.161528,3,Metadata:\nDimensions: T(6) x XY(9) x Z(7) x λ...,D06,TRITC,ARMC5,1
99,2304,2304,Plan Apo VC 20x DIC N2,0.325,1,1,WellD06_ChannelTRITC_Seq0007_0001.ome.tiff,8134.297253,-5617.271429,3204.714286,7,2024-08-02 22:05:33.297253,3,Metadata:\nDimensions: T(38) x XY(9) x Z(7) x ...,D06,TRITC,ARMC5,1
441,2304,2304,Plan Apo VC 20x DIC N2,0.325,1,1,WellD06_ChannelTRITC_Seq0009_0001.ome.tiff,8265.229409,-5617.185714,6004.300000,7,2024-08-02 22:09:55.229409,3,Metadata:\nDimensions: T(38) x XY(9) x Z(7) x ...,D06,TRITC,ARMC5,1
783,2304,2304,Plan Apo VC 20x DIC N2,0.325,1,1,WellD06_ChannelTRITC_Seq0001_0001.ome.tiff,930.540839,-5616.985714,3204.614286,7,2024-08-02 10:26:30.540839,3,Metadata:\nDimensions: T(6) x XY(9) x Z(7) x λ...,D06,TRITC,ARMC5,1
1701,2304,2304,Plan Apo VC 20x DIC N2,0.325,1,1,WellD05_ChannelTRITC_Seq0008_0001.ome.tiff,8199.607210,-5611.357143,14947.100000,7,2024-08-02 22:07:43.607210,3,Metadata:\nDimensions: T(38) x XY(9) x Z(7) x ...,D05,TRITC,Scrambled,1
2043,2304,2304,Plan Apo VC 20x DIC N2,0.325,1,1,WellD05_ChannelTRITC_Seq0000_0001.ome.tiff,865.631245,-5611.242857,12147.385714,7,2024-08-02 10:24:20.631245,3,Metadata:\nDimensions: T(6) x XY(9) x Z(7) x λ...,D05,TRITC,Scrambled,1
2097,2304,2304,Plan Apo VC 20x DIC N2,0.325,1,1,WellD05_ChannelTRITC_Seq0006_0001.ome.tiff,8069.361694,-5611.328571,12147.400000,7,2024-08-02 22:03:24.361694,3,Metadata:\nDimensions: T(38) x XY(9) x Z(7) x ...,D05,TRITC,Scrambled,1
2439,2304,2304,Plan Apo VC 20x DIC N2,0.325,1,1,WellD05_ChannelTRITC_Seq0002_0001.ome.tiff,995.638590,-5611.371429,14947.200000,7,2024-08-02 10:28:41.638590,3,Metadata:\nDimensions: T(6) x XY(9) x Z(7) x λ...,D05,TRITC,Scrambled,1


In [27]:
after_bleach_images = ["WellD05_ChannelTRITC_Seq0006_0001.ome.tiff", "WellD06_ChannelTRITC_Seq0007_0001.ome.tiff"]
for f in after_bleach_images:
    aics_image = AICSImage(Path(image_dir) / f, reader=readers.ome_tiff_reader.OmeTiffReader)
    mCherry_array = aics_image.get_image_data('YX',Z=0,C=0,T=0)
    mCherry_image = illumination_correction.correct(AICSImage(mCherry_array[np.newaxis,np.newaxis,np.newaxis,:,:]))
    # crop 500 pixels from left edge
    mCherry_image = AICSImage(mCherry_image.get_image_data('YX',Z=0,C=0,T=0)[np.newaxis,np.newaxis,np.newaxis,:,500:])
    mCherry_image.save(str(training_dir / f))

## Export images for example bleach-chase movie

In [6]:
movie_dir = Path(image_dir).parent / 'MOVIE'
if not movie_dir.exists(): 
    movie_dir.mkdir()

pre_bleach_images = ["WellD05_ChannelTRITC_Seq0000_0005.ome.tiff", "WellD06_ChannelTRITC_Seq0001_0005.ome.tiff"]
after_bleach_images = ["WellD05_ChannelTRITC_Seq0006_0005.ome.tiff", "WellD06_ChannelTRITC_Seq0007_0005.ome.tiff"]
for f in (pre_bleach_images + after_bleach_images):
    aics_image = AICSImage(Path(image_dir) / f, reader=readers.ome_tiff_reader.OmeTiffReader)
    mCherry_array = aics_image.get_image_data('TYX',Z=0,C=0)
    mCherry_image = illumination_correction.correct(AICSImage(mCherry_array[:,np.newaxis,np.newaxis,:,:]))
    # crop 500 pixels from left edge, 100 pixels from right edge and 300 from top and bottom
    mCherry_image = AICSImage(mCherry_image.get_image_data('TYX',Z=0,C=0)[:,np.newaxis,np.newaxis,300:1748,500:1948])
    mCherry_image.save(str(movie_dir / f))

In [7]:
movie_dir = Path(image_dir).parent / 'MOVIE'
if not movie_dir.exists(): 
    movie_dir.mkdir()

pre_bleach_images = ["WellD05_ChannelTRITC_Seq0002_0005.ome.tiff", "WellD06_ChannelTRITC_Seq0003_0005.ome.tiff"]
after_bleach_images = ["WellD05_ChannelTRITC_Seq0008_0005.ome.tiff", "WellD06_ChannelTRITC_Seq0009_0005.ome.tiff"]
for f in (pre_bleach_images + after_bleach_images):
    aics_image = AICSImage(Path(image_dir) / f, reader=readers.ome_tiff_reader.OmeTiffReader)
    mCherry_array = aics_image.get_image_data('TYX',Z=0,C=0)
    mCherry_image = illumination_correction.correct(AICSImage(mCherry_array[:,np.newaxis,np.newaxis,:,:]))
    # crop 500 pixels from left edge, 100 pixels from right edge and 300 from top and bottom
    mCherry_image = AICSImage(mCherry_image.get_image_data('TYX',Z=0,C=0)[:,np.newaxis,np.newaxis,300:1748,500:1948])
    mCherry_image.save(str(movie_dir / f))